In [2]:
import cv2
import mediapipe as mp
import pygame
import math

# Initialize PyGame Mixer
pygame.mixer.init()

# Load a music file
def load_music(file_path):
    pygame.mixer.music.load(file_path)

# Play music
def play_music():
    pygame.mixer.music.play()

# Pause music
def pause_music():
    pygame.mixer.music.pause()

# Stop music
def stop_music():
    pygame.mixer.music.stop()

# Next song (can be handled by changing the file being loaded)
def next_song(song_list, current_index):
    if current_index < len(song_list) - 1:
        current_index += 1
        load_music(song_list[current_index])
        play_music()
    return current_index

# Previous song
def previous_song(song_list, current_index):
    if current_index > 0:
        current_index -= 1
        load_music(song_list[current_index])
        play_music()
    return current_index

# Volume up
def volume_up():
    volume = pygame.mixer.music.get_volume()
    pygame.mixer.music.set_volume(min(1.0, volume + 0.1))
    print(f"Volume up: {pygame.mixer.music.get_volume()}")

# Volume down
def volume_down():
    volume = pygame.mixer.music.get_volume()
    pygame.mixer.music.set_volume(max(0.0, volume - 0.1))
    print(f"Volume down: {pygame.mixer.music.get_volume()}")

# Favorite song (you can implement by marking it in a list or database)
def favorite_song(song):
    print(f"{song} added to favorites.")

# Add song (append to playlist)
def add_song(song_list, song):
    song_list.append(song)

# Delete song (remove from playlist)
def delete_song(song_list, song):
    if song in song_list:
        song_list.remove(song)

# Search song
def search_song(song_list, query):
    results = [song for song in song_list if query.lower() in song.lower()]
    print(f"Search results: {results}")
    return results

# Helper function to calculate the distance between two landmarks
def calculate_distance(point1, point2):
    return math.sqrt((point1.x - point2.x) ** 2 + (point1.y - point2.y) ** 2 + (point1.z - point2.z) ** 2)

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Gesture recognition logic with more gestures
def recognize_gesture(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    thumb_ip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP]
    thumb_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_CMC]  # Thumb base (near wrist)
    wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]
    
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
    
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    middle_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]

    ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    ring_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP]

    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
    pinky_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP]

    # Calculate distances for gesture recognition
    thumb_wrist_dist = calculate_distance(thumb_tip, wrist)  # Thumb distance from wrist
    index_finger_bent = calculate_distance(index_tip, index_mcp) < 0.04  # Check if index finger is curled
    middle_finger_bent = calculate_distance(middle_tip, middle_mcp) < 0.04  # Check if middle finger is curled
    ring_finger_bent = calculate_distance(ring_tip, ring_mcp) < 0.04  # Check if ring finger is curled
    pinky_finger_bent = calculate_distance(pinky_tip, pinky_mcp) < 0.04  # Check if pinky finger is curled

    # Check if thumb is extended
    thumb_extended = thumb_wrist_dist > 0.12  # Adjust threshold based on your data

    # Calculate thumb angle to detect thumbs up/down
    thumb_angle = math.degrees(math.atan2(thumb_tip.y - wrist.y, thumb_tip.x - wrist.x))

    # Gesture 1: Open Palm (play)
    if not index_finger_bent and not middle_finger_bent and not ring_finger_bent and not pinky_finger_bent:
        return "play"
    
    # Gesture 2: Fist (pause)
    if index_finger_bent and middle_finger_bent and ring_finger_bent and pinky_finger_bent:
        return "pause"
    
    # Gesture 3: Thumbs up (volume up)
    if thumb_extended and index_finger_bent and middle_finger_bent and ring_finger_bent and pinky_finger_bent and thumb_angle > 0:
        return "volume up"
    
    # Gesture 4: Thumbs down (volume down)
    if thumb_extended and index_finger_bent and middle_finger_bent and ring_finger_bent and pinky_finger_bent and thumb_angle < 0:
        return "volume down"

    # Gesture 5: Two fingers (next song) - Index and middle extended
    if not index_finger_bent and not middle_finger_bent and ring_finger_bent and pinky_finger_bent:
        return "next song"

    # Gesture 6: Index and pinky extended (previous song)
    if not index_finger_bent and middle_finger_bent and ring_finger_bent and not pinky_finger_bent:
        return "previous song"

    return None  # No recognized gesture

# Define song playlist
song_list = ['Music/Playlists/Anne_Marie_-_2002_(Jesusful.com).mp3', 'Music/Playlists/MIYEON   The Painted On The Moonlight.mp3']  # Your actual song files
current_index = 0
load_music(song_list[current_index])

# Start webcam feed
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_lms in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(img, hand_lms, mp_hands.HAND_CONNECTIONS)

            # Recognize the gesture
            gesture = recognize_gesture(hand_lms)
            if gesture:
                print(f"Recognized Gesture: {gesture}")

                # Call music control functions based on recognized gesture
                if gesture == "play":
                    play_music()
                elif gesture == "pause":
                    pause_music()
                elif gesture == "volume up":
                    volume_up()
                elif gesture == "volume down":
                    volume_down()
                elif gesture == "next song":
                    current_index = next_song(song_list, current_index)
                elif gesture == "previous song":
                    current_index = previous_song(song_list, current_index)

    cv2.imshow("Hand Gesture Recognition", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Recognized Gesture: next song
Recognized Gesture: next song
Recognized Gesture: next song
Recognized Gesture: next song
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
Recognized Gesture: pause
